# 🚀 Training YOLOv8 on Amazon SageMaker
**Project:** PPE Detection (Object Detection)

In this notebook, we orchestrate the training of a **YOLOv8** model using AWS SageMaker infrastructure.

### The Strategy: "Bring Your Own Script" (BYOS)
SageMaker does not have a built-in YOLOv8 container. We will use the standard **PyTorch 2.8** container and inject a custom script (`train_yolo.py`) that:
1. Installs the `ultralytics` library at runtime.
2. Dynamically rewrites the dataset paths (`data.yaml`) to match the container's file system.
3. Fine-tunes the model and exports the weights (`best.pt`) to S3.

## Setup

In [4]:
import sagemaker
import boto3
import os
import urllib.request
import zipfile
import yaml
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

boto_session = boto3.Session(profile_name='pessoal', region_name='us-east-1')
bucket_name = 'sagemaker-portfolio-cv-aws' 
sess = sagemaker.Session(boto_session=boto_session, default_bucket=bucket_name)
region = sess.boto_region_name

print(f"Região: {region}")
print(f"Bucket alvo: {bucket_name}")

Região: us-east-1
Bucket alvo: sagemaker-portfolio-cv-aws


In [17]:
import sys
sys.path.append(os.path.abspath(".."))
from config_private import IAM_ROLE
role = IAM_ROLE

## Data Location
We point to the S3 bucket where we uploaded the PPE dataset in the previous notebook.

In [6]:
# The prefix used in notebook 01
prefix = 'data/raw/ppe'
s3_input_data = f's3://{bucket_name}/{prefix}'

print(f"Training data will be downloaded from: {s3_input_data}")

Training data will be downloaded from: s3://sagemaker-portfolio-cv-aws/data/raw/ppe


## Hyperparameters & Configuration
We define the training parameters. 
* **`epochs`**: Number of passes through the dataset.
* **`imgsz`**: Image resolution (YOLO standard is 640).
* **`model-name`**: `yolov8n.pt` (Nano) for speed, but you can switch to `yolov8m.pt` (Medium) for accuracy.

In [12]:
from sagemaker.pytorch import PyTorch



hyperparameters = {
    'epochs': 1,
    'batch-size': 8,
    'imgsz': 640,
    'model-name': 'yolov8n.pt',
    'device': 'cpu'
}

# Metric Definitions
metric_definitions = [
    {'Name': 'mAP50', 'Regex': r'mAP50\s+([0-9\.]+)'},
    {'Name': 'mAP50-95', 'Regex': r'mAP50-95\s+([0-9\.]+)'}
]

# The Estimator
estimator = PyTorch(
    entry_point='train_yolo.py',
    source_dir='../../sagemaker_entry_point/ppe',
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',   
    framework_version='2.8.0',
    py_version='py312',
    hyperparameters=hyperparameters,
    metric_definitions=metric_definitions,
    sagemaker_session=sess
)

## Launch Training 
This process triggers the following workflow:
1. **Provisioning:** AWS spins up the EC2 instance.
2. **Downloading:** The PPE dataset is copied from S3 to `/opt/ml/input/data/training`.
3. **Setup:** The container installs `ultralytics`.
4. **Training:** YOLOv8 runs for the specified epochs.
5. **Upload:** The final model (`model.pt`) is zipped and sent to S3.

In [13]:
estimator.fit({'training': s3_input_data})

INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: pytorch-training-2025-11-26-17-54-10-326


2025-11-26 17:54:15 Starting - Starting the training job...
2025-11-26 17:54:50 Downloading - Downloading input data...
2025-11-26 17:55:21 Downloading - Downloading the training image.....bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2025-11-26 17:56:13,662 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2025-11-26 17:56:13,662 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-11-26 17:56:13,663 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2025-11-26 17:56:13,672 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2025-11-26 17:56:13,685 sagemaker_pytorch_container.training INFO     Invoking user training script.
2025-11-26 17:56:14,822 sagemaker-training-toolkit INFO     Provided path: /opt/ml/code  is empty, unzipping
2025-11-26 17:56:15,157 sagemaker-trainin